In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-05-05"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
10329,2024-05-05,Espanha Acb,07:30,Zaragoza,Manresa,1.99,1.82,172.5,1.88,1.92,1.5,1.86,1.69,ncp0tYiA,0.502513,0.549451,0.531915,0.520833,0.051963,0.6,0.6,NaN,NaN,166.614,55.541078,0.333352,177.746,26.869376,0.151167,145.644,254.714,147.00,174.84,0.0,0.0,0.0,0.0,0.063101,0.014886,0.067723,-0.74,-0.148,-6.689189,0.543117,0.7,0.156883,9.56,1.912,0.428870,0.366899,0.6,0.233101
10330,2024-05-05,Eua Nba,14:00,Cleveland Cavaliers,Orlando Magic,1.61,2.41,194.5,1.93,1.93,-4.5,1.95,3.30,EihMmTT9,0.621118,0.414938,0.518135,0.518135,0.036056,1.0,0.6,NaN,NaN,192.686,29.624549,0.153745,206.662,42.584236,0.206057,187.380,194.196,157.04,275.01,0.0,0.0,0.0,0.0,0.281436,0.000000,0.363655,0.02,0.004,152.500000,0.664136,0.6,-0.064136,-5.00,-1.000,-1.410000,0.512458,0.3,-0.212458
10331,2024-05-05,Itália Liga A,13:15,Sassari,Reggiana,2.00,1.81,163.5,1.83,1.97,1.5,1.86,1.67,IyVeoWRg,0.500000,0.552486,0.546448,0.507614,0.052486,0.8,0.2,NaN,NaN,170.076,36.891545,0.216912,182.140,71.655449,0.393409,171.738,177.180,123.84,161.54,0.0,0.0,0.0,0.0,0.070525,0.052103,0.076119,2.78,0.556,1.798561,0.000000,0.0,0.000000,-3.22,-0.644,-1.257764,0.453260,0.2,-0.253260
10332,2024-05-05,Itália Liga A,13:15,Treviso,Tortona,1.69,2.19,164.5,1.85,1.95,-3.5,2.00,2.48,CzO4qAd6,0.591716,0.456621,0.540541,0.512821,0.048337,0.4,0.8,NaN,NaN,210.878,54.658117,0.259193,139.190,20.409900,0.146633,170.684,189.224,150.52,157.14,0.0,0.0,0.0,0.0,0.182244,0.037216,0.151523,-1.00,-0.200,-3.450000,0.504576,0.4,-0.104576,-3.28,-0.656,-1.814024,0.455289,0.2,-0.255289
10333,2024-05-05,Alemanha Pro A,11:30,Jena,Frankfurt,1.69,2.06,153.5,1.85,1.85,-2.5,1.92,2.41,SW3iDlM8,0.591716,0.485437,0.540541,0.540541,0.077153,0.6,0.4,NaN,NaN,122.232,34.030326,0.278408,117.204,26.179921,0.223371,127.938,113.852,98.04,143.50,0.0,0.0,0.0,0.0,0.139536,0.000000,0.160038,0.41,0.082,8.414634,0.000000,0.0,0.000000,0.84,0.168,6.309524,0.000000,0.0,0.000000
10334,2024-05-05,Alemanha Pro A,12:00,Kirchheim,Phoenix Hagen,2.01,1.73,160.5,1.86,1.84,1.5,1.86,1.60,AFLsO5zR,0.497512,0.578035,0.537634,0.543478,0.075547,0.4,0.6,NaN,NaN,189.394,55.556853,0.293340,125.352,13.785946,0.109978,141.468,138.976,146.52,140.76,0.0,0.0,0.0,0.0,0.105877,0.007644,0.106270,-1.16,-0.232,-4.353448,0.628643,0.7,0.071357,-1.63,-0.326,-2.239264,0.573798,0.5,-0.073798
10335,2024-05-05,Alemanha Pro A,13:00,PS Karlsruhe,Giessen,1.75,1.98,167.5,1.85,1.85,-2.5,2.01,2.39,lUbUE1cD,0.571429,0.505051,0.540541,0.540541,0.076479,0.6,0.6,NaN,NaN,166.952,40.064463,0.239976,140.098,27.211989,0.194235,127.164,149.758,124.20,158.79,0.0,0.0,0.0,0.0,0.087204,0.000000,0.122137,-0.34,-0.068,-11.029412,0.000000,0.0,0.000000,-0.60,-0.120,-8.166667,0.000000,0.0,0.000000
10336,2024-05-05,Argentina Liga A,21:00,Ferro,Obras,1.98,1.75,166.5,1.86,1.84,1.5,1.84,1.61,xdhEoItq,0.505051,0.571429,0.537634,0.543478,0.076479,0.6,0.6,NaN,NaN,157.122,53.890211,0.342983,175.678,40.435628,0.230169,134.916,208.148,142.80,234.00,0.0,0.0,0.0,0.0,0.087204,0.007644,0.094281,1.65,0.330,2.969697,0.000000,0.0,0.000000,-0.71,-0.142,-5.281690,0.465913,0.5,0.034087
10337,2024-05-05,Austrália Nbl1 Leste,00:00,Hills Hornets,Canberra,2.07,1.70,159.5,1.85,1.85,-2.5,1.97,1.89,KpDA2iJi,0.483092,0.588235,0.540541,0.540541,0.071327,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.138795,0.000000,0.029310,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
10338,2024-05-05

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,14:00,Eua Nba,Cleveland Cavaliers,Orlando Magic,194.5,1.93,1.0000,Over
1,13:15,Itália Liga A,Treviso,Tortona,164.5,1.85,1.0000,Over
2,11:30,Alemanha Pro A,Jena,Frankfurt,153.5,1.85,1.0000,Over
3,13:00,Alemanha Pro A,PS Karlsruhe,Giessen,167.5,1.85,1.0000,Over
4,00:00,Austrália Nbl1 Leste,Hills Hornets,Canberra,159.5,1.85,0.8890,Over
5,02:30,Austrália Nbl1 Leste,Sutherland,Manly W.,169.5,1.85,0.9991,Over
6,04:00,Filipinas Copa Das Filipinas,Magnolia Hotshots,TNT Tropang Giga,185.5,1.87,1.0000,Over
7,10:30,França Lfb Feminina,Flammes Carolo F,Landerneau Bretagne F,143.5,1.86,1.0000,Over
8,02:05,Japão B.League,Osaka,Shimane,160.5,1.80,1.0000,Over
9,03:05,Japão B.League,SeaHorses Mikawa,NeoPhoenix,162.5,1.83,0.9968,Over
